Model Building - Regression Template

In [ ]:
import pandas as pd
import numpy as np
import joblib
import warnings
warnings.filterwarnings('ignore')

from config import modelling_file_path, model_output_path, prediction_output_path
from src.modelling_data_contract import ModellingDataContract
from src.modelling.hyperparameter_tuning import XGBHyperparameterTuner
from src.modelling.supermodel import SuperXGBRegressor
from src.modelling.optuna_xgb_param_grid import OptunaXGBParamGrid

pd.options.display.max_rows = 100
pd.options.display.max_columns = 999

%load_ext autoreload
%autoreload 2

Model Version

In [ ]:
model_version = 1
model_name = ''
model_file_name = model_name + '_v' + str(model_version)

In [ ]:
RESPONSE = ModellingDataContract.RESPONSE

In [ ]:
FEATURES = ModellingDataContract.feature_list

In [ ]:
MONOTONE_CONSTRAINTS = ModellingDataContract.monotone_constraints

Load Data

In [ ]:
modelling_data = pd.read_csv(modelling_file_path)
modelling_data.tail()

In [ ]:
training_data = modelling_data[(modelling_data[RESPONSE+"TrainingSet"]) | (modelling_data[RESPONSE+"ValidationSet"])]
test_data = modelling_data[modelling_data[RESPONSE+"TestSet"]]

In [ ]:
X, y = modelling_data.drop(columns=[RESPONSE]), modelling_data[RESPONSE]
X_train, y_train = training_data.drop(columns=[RESPONSE]), training_data[RESPONSE]
X_test, y_test = test_data.drop(columns=[RESPONSE]), test_data[RESPONSE]

In [ ]:
X_preproc = X[FEATURES]
X_train_preproc = X_train[FEATURES]
X_test_preproc = X_test[FEATURES]

Optuna Hyperparameter Tuning Class - HyperParameterTuner & XGBHyperparameterTuner

In [ ]:
xgb_tuner = XGBHyperparameterTuner(X_train_preproc, y_train, monotonicity_constraints=MONOTONE_CONSTRAINTS)

In [ ]:
xgb_tuner.tune_hyperparameters()

In [ ]:
params = xgb_tuner.get_best_params()
params

Training Model - SuperXGBClassifier class for training and predictions

In [ ]:
params['objective'] = OptunaXGBParamGrid.error
params['num_rounds'] = 1000
params['early_stopping_rounds'] = 50
params['verbosity'] = 1
params['monotone_constraints'] = MONOTONE_CONSTRAINTS

In [ ]:
super_xgb = SuperXGBRegressor(X_train = X_train_preproc, 
                              y_train = y_train, 
                              X_test = X_test_preproc, 
                              y_test = y_test,
                              params = params)

In [ ]:
super_xgb.fit()

In [ ]:
super_xgb.xgb_model

In [ ]:
super_xgb.xgb_model.get_booster().feature_names

In [ ]:
preds = super_xgb.predict(X_preproc)
train_preds = super_xgb.predict(X_train_preproc)
test_preds = super_xgb.predict(X_test_preproc)

Check Average Predictions

In [ ]:
preds.mean(), modelling_data[RESPONSE].mean()

In [ ]:
train_preds.mean(), training_data[RESPONSE].mean()

In [ ]:
test_preds.mean(), test_data[RESPONSE].mean()

Export model

In [ ]:
super_xgb.export_model(model_output_path + "/" + model_file_name + ".joblib")

Export data and predictions

In [ ]:
modelling_info = modelling_data[['match_id', "chain_number", "order", RESPONSE]].reset_index(drop = True)
modelling_export = pd.concat([modelling_info, X_preproc.reset_index(drop=True)], axis=1)
modelling_export['xgb_preds'] = preds
modelling_export.to_csv(prediction_output_path + 'predictions_' + model_file_name + '.csv', index = False)
modelling_export.head()